# Debias BERTJe

In [21]:
# import torch
%load_ext autoreload
# assert(1==2) #dont use
#dont use
%autoreload 2
from transformers import *
import numpy as np
import pandas as pd
import os
import SEAT
import Utils_R as utils_r
import Debias_Sentences as sen_debias

# Main process steps 

# Debias BERT Steps

1. Initialize BERT
2. Get Gender Direction
    1. Get lists of pair sentences. (e.g. HE is smart, SHE is smart).
    2. Get embeddings on pair list.
    3. Normalize embeddings.
    4. Get component 0 of PCA(pairs_list).
    5. return that component 0.
3. Debias some sentences
    1. Get sentence embeddings
    2. Normalize embeddings
    3. Use drop method with gender Direction
    4. Normalize again
4. Perform SEAT (same as step 3 but sentences are from seat)

In [2]:
tokenizer_nl = RobertaTokenizer.from_pretrained("pdelobelle/robBERT-base")
model_nl = RobertaModel.from_pretrained("pdelobelle/robBERT-base")
dir_results = '../Rodrigo-data/Results/'

# Biased

In [3]:
df_biased = SEAT.Compute_SEAT(model_nl, tokenizer_nl, model_name = 'ROBBERT_BIAS',output_results='dutch_biased_robbert.csv',from_cache=False,from_cache_XYAB_=False)

100%|██████████| 47/47 [00:01<00:00, 24.34it/s]


# Debiased

In [4]:
k_dim_file = 50
pairs_aumount = 30000
#Get bias subspace
file_bias_subspace = "../Rodrigo-data/bias_subspace/Robbert_ForFinetune-bias_subspace_k50_nl_large30000k"
bias_dir = sen_debias.get_bias_direction(model_nl,tokenizer_nl,from_cache = True,filename_cache= file_bias_subspace)
# SEAT test with debaised direction
k_dim = 1    #from how many dim we want to debias
out_results = 'Results_debiased/RobBert_new_debiased_results_k'+str(k_dim)+'_from_'+str(pairs_aumount)+'_pairs_nl.csv' 
df_debiased = SEAT.Compute_SEAT(model_nl, tokenizer_nl, bias_dir = bias_dir, k_dimensions = k_dim,
                                model_name = 'ROBBERT_DEBIAS', output_results=out_results,
                                cache_file=out_results, from_cache=False,from_cache_XYAB_=False)

100%|██████████| 47/47 [00:01<00:00, 24.85it/s]


### Mean Effect Size

In [5]:
 mean_effect_size, mean_p_val = SEAT.get_means_results(df_debiased)
 print('Mean effect size with '+str(k_dim)+' k-dim')
 print(mean_effect_size)
 print(mean_p_val) 

Mean effect size with 1 k-dim
0.3025721887747447
0.5066666666666666


### LaTeX Results

In [23]:
latex_bert = utils_r.create_latex_table(df_biased,df_debiased)
print(latex_bert)

\begin{table}[htb!]
    \begin{center}

    \begin{tabular}{|c|c|c|}
    \hline
        SEAT list &  Effect size d &  Significance p \\
       \hline
    SEAT-01 & $-0.2045  \rightarrow \textbf{-0.2114}$ & $0.0 \rightarrow 0.0000 $ \\
 \hline
 SEAT-02 & $0.1307  \rightarrow 0.1343$ & $0.04 \rightarrow 0.0400 $ \\
 \hline
 SEAT-03 & $0.1193  \rightarrow \textbf{0.0932}$ & $0.09 \rightarrow \textbf{0.2000} $ \\
 \hline
 SEAT-03b & $0.0254  \rightarrow 0.032$ & $0.41 \rightarrow 0.2600 $ \\
 \hline
 SEAT-04 & $0.0714  \rightarrow \textbf{0.0673}$ & $0.32 \rightarrow 0.2200 $ \\
 \hline
 SEAT-05 & $0.6794  \rightarrow \textbf{0.6756}$ & $0.01 \rightarrow 0.0100 $ \\
 \hline
 SEAT-05b & $0.0978  \rightarrow 0.1014$ & $0.14 \rightarrow 0.1000 $ \\
 \hline
 SEAT-06 & $-0.1914  \rightarrow -0.1904$ & $0.12 \rightarrow \textbf{0.1800} $ \\
 \hline
 SEAT-06b & $-0.5519  \rightarrow \textbf{-0.5846}$ & $0.0 \rightarrow 0.0000 $ \\
 \hline
 SEAT-07 & $-0.3308  \rightarrow -0.2518$ & $0.03 \rightar

In [24]:
import Utils_R_DELETEIFYOUSEETHIS_TEMPFILE as ut_3

In [25]:
latex_bert = ut_3.create_latex_table(df_biased,df_debiased)
print(latex_bert)

\begin{table}[htb!]
    \begin{center}

    \begin{tabular}{|c|c|c|}
    \hline
        SEAT list &  Effect size d &  Significance p \\
       \hline
    SEAT-01 & $0.2045  \rightarrow 0.2114$ & $0.0 \rightarrow 0.0000 $ \\
 \hline
 SEAT-02 & $0.1307  \rightarrow 0.1343$ & $0.04 \rightarrow 0.0400 $ \\
 \hline
 SEAT-03 & $0.1193  \rightarrow \textbf{0.0932}$ & $0.09 \rightarrow \textbf{0.2000} $ \\
 \hline
 SEAT-03b & $0.0254  \rightarrow 0.032$ & $0.41 \rightarrow 0.2600 $ \\
 \hline
 SEAT-04 & $0.0714  \rightarrow \textbf{0.0673}$ & $0.32 \rightarrow 0.2200 $ \\
 \hline
 SEAT-05 & $0.6794  \rightarrow \textbf{0.6756}$ & $0.01 \rightarrow 0.0100 $ \\
 \hline
 SEAT-05b & $0.0978  \rightarrow 0.1014$ & $0.14 \rightarrow 0.1000 $ \\
 \hline
 SEAT-06 & $0.1914  \rightarrow \textbf{0.1904}$ & $0.12 \rightarrow \textbf{0.1800} $ \\
 \hline
 SEAT-06b & $0.5519  \rightarrow 0.5846$ & $0.0 \rightarrow 0.0000 $ \\
 \hline
 SEAT-07 & $0.3308  \rightarrow \textbf{0.2518}$ & $0.03 \rightarrow \tex